# T5 + doc2query - Indexing

Author: Monique Monteiro (moniquelouise@gmail.com)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


## Libraries installation

In [ ]:
!pip install pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pyserini-0.21.0-py3-none-any.whl (154.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 28.0 MB/s eta 0:00:00
  Using cached sentencepiece-0.1.97-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached pyjnius-1.4.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.5 MB)
  Using cached nmslib-2.1.1-cp39-cp39-manylinux2010_x86_64.whl (13.3 MB)
  Using cached onnxruntime-1.14.1-cp39-cp39-manylinux_2_27_x86_64.whl (5.0 MB)
  Using cached transformers-4.27.4-py3-none-any.whl (6.8 MB)
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 kB 36.8 MB/s eta 0:00:00
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl

In [ ]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━

In [ ]:
!pip install -q condacolab

In [ ]:
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
!conda install faiss-cpu -c pytorch

Solving environment: / - \ | / - \ | / - \ | / - \ | / done

# All requested packages already installed.



In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install trectools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Index building

In [ ]:
from pyserini.search.lucene import LuceneSearcher

In [ ]:
#beir-v1.0.0-trec-covid.flat
LuceneSearcher.from_prebuilt_index('beir-v1.0.0-trec-covid.flat')

lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.tar.gz: 216MB [00:23, 9.51MB/s]                           


In [ ]:
!ls ~/.cache/pyserini/indexes/

lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade


## Dataset download and preparation



In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/queries.jsonl.gz

--2023-04-10 22:42:37--  https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/queries.jsonl.gz
Resolving huggingface.co (huggingface.co)... 13.249.85.69, 13.249.85.127, 13.249.85.92, ...
Connecting to huggingface.co (huggingface.co)|13.249.85.69|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/a8/10/a810e88b0e7b233be82b89c1fa6ec2d75efc6d55784c2ada9dcac8434a634f3a/9eadcc2cdf140addc9dae83648bb2c6611f5e4b66eaed7475fa5a0ca48eda371?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27queries.jsonl.gz%3B+filename%3D%22queries.jsonl.gz%22%3B&response-content-type=application%2Fgzip&Expires=1681425758&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2E4LzEwL2E4MTBlODhiMGU3YjIzM2JlODJiODljMWZhNmVjMmQ3NWVmYzZkNTU3ODRjMmFkYTlkY2FjODQzNGE2MzRmM2EvOWVhZGNjMmNkZjE0MGFkZGM5ZGFlODM2NDhiYjJjNjYxMWY1ZTRiNjZlYWVkNzQ3NWZhNWEwY2E0OGVkYTM3MT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9u

In [ ]:
main_dir = "/content/gdrive/MyDrive/Unicamp-aula-6-3"

In [ ]:
!mv queries.jsonl.gz {main_dir}/trec-covid

In [ ]:
!gunzip {main_dir}/trec-covid/queries.jsonl.gz

gzip: /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.jsonl already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [ ]:
import json

with open(f'{main_dir}/trec-covid/queries.tsv','w') as f_out:
  with open(f'{main_dir}/trec-covid/queries.jsonl', 'r') as f:
    for line in f:
      data = json.loads(line)
      id = data['_id']
      text = data['text']
      f_out.write(f'{id}\t{text}\n')
        


In [ ]:
topics = "trec-covid"

In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv

--2023-04-10 22:44:42--  https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv
Resolving huggingface.co (huggingface.co)... 13.249.85.69, 13.249.85.16, 13.249.85.127, ...
Connecting to huggingface.co (huggingface.co)|13.249.85.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 980831 (958K) [text/plain]
Saving to: ‘test.tsv’

test.tsv            100%[===================>] 957.84K  --.-KB/s    in 0.07s   

2023-04-10 22:44:42 (13.7 MB/s) - ‘test.tsv’ saved [980831/980831]



In [ ]:
!mv test.tsv {main_dir}/trec-covid

In [ ]:
import pandas as pd

qrel = pd.read_csv(f"{main_dir}/trec-covid/test.tsv", sep="\t", header=None, 
                   skiprows=1, names=["query", "docid", "rel"])
qrel["q0"] = "q0"
qrel = qrel.to_dict(orient="list")

## Searching with BM25 the original index

In [ ]:
!python -m pyserini.search.lucene \
  --index ~/.cache/pyserini/indexes/lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned.txt \
  --hits 1000 \
  --bm25 --k1 0.82 --b 0.68

Setting BM25 parameters: k1=0.82, b=0.68
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned.txt...
100% 50/50 [00:06<00:00,  8.18it/s]


In [ ]:
run = pd.read_csv(f"{main_dir}/runs/run.{topics}.bm25tuned.txt", sep="\s+", 
                  names=["query", "q0", "docid", "rank", "score", "system"])
run = run.to_dict(orient="list")

In [ ]:
from evaluate import load
trec_eval = load("trec_eval")
results = trec_eval.compute(predictions=[run], references=[qrel])

In [ ]:
results['NDCG@10']

0.5963435398557583

## Searching with BM25 the expanded index

In [36]:
#index_name = "expanded_trec_covid"
index_name = "expanded_trec_covid_3_queries"
#index_name = "expanded_trec_covid_20_queries"

In [44]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name} \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}.txt \
  --hits 1000 \
  --bm25 --k1 0.82 --b 0.68

Setting BM25 parameters: k1=0.82, b=0.68
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-expanded_trec_covid_3_queries.txt...
100% 50/50 [01:02<00:00,  1.26s/it]


In [45]:
run = pd.read_csv(f"{main_dir}/runs/run.{topics}.bm25tuned-{index_name}.txt", sep="\s+", 
                  names=["query", "q0", "docid", "rank", "score", "system"])
run = run.to_dict(orient="list")

In [46]:
from evaluate import load
trec_eval = load("trec_eval")
results = trec_eval.compute(predictions=[run], references=[qrel])

In [47]:
results['NDCG@10']

0.6540338139566075

In [ ]:
#NDCTG@10 for run generated by doc2query-adafactor-bs-32-split-1000-no-fp16
from evaluate import load

run = pd.read_csv(f"{main_dir}/runs/run.{topics}.bm25tuned-expanded_doc2query-adafactor-bs-32-split-1000-no-fp16.txt", sep="\s+", 
                  names=["query", "q0", "docid", "rank", "score", "system"])
run = run.to_dict(orient="list")

trec_eval = load("trec_eval")
results = trec_eval.compute(predictions=[run], references=[qrel])

In [ ]:
results['NDCG@10']

0.6440970227395291

## Removal of documents without text

In [ ]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import jsonlines

corpus = []
docs_to_remove_expansion = dict()

with jsonlines.open(f"{main_dir}/trec-covid/corpus.jsonl") as reader:
    for item in reader:
        if len(item["text"].strip()) == 0:
          docs_to_remove_expansion[item["_id"]] = item["title"]

In [ ]:
len(docs_to_remove_expansion)

42140

In [48]:
#index_name = "trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping"
#index_name = "trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_20_queries"
index_name = "trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries"

In [49]:
collections_path = f"{main_dir}/collections/{index_name}"
collections_path

'/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries'

In [53]:
import glob
import os
from tqdm.auto import tqdm

def remove_expansions_from_empty_docs(index_name):
  collections_path = f"{main_dir}/collections/{index_name}"
  new_collections_path = collections_path + "-expand-non-empty"
  extension = "*.json"
  files = glob.glob(os.path.join(collections_path, extension))
  print(files)

  for file in tqdm(files):
    with jsonlines.open(file) as reader:
      # Loop through each line in the file
      itens = []
      for item in reader:
        if item["id"] in docs_to_remove_expansion:
          itens.append({"id":item["id"], "contents":docs_to_remove_expansion[item["id"]]})
        else:
          itens.append(item)
      with jsonlines.open(new_collections_path + "/" + file[file.rfind('/')+1:], mode='w') as writer:
        for item in itens:
          writer.write(item)
        print(new_collections_path + "/" + file[file.rfind('/')+1:])

  return new_collections_path

In [54]:
new_collections_path = remove_expansions_from_empty_docs(index_name)

['/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries/json_10.json', '/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries/json_1.json', '/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries/json_3.json', '/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries/json_2.json', '/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries/json_0.json', '/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries/json_5.json', '/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queri

  0%|          | 0/11 [00:00<?, ?it/s]

/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries-expand-non-empty/json_10.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries-expand-non-empty/json_1.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries-expand-non-empty/json_3.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries-expand-non-empty/json_2.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries-expand-non-empty/json_0.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries-expand-non-empty/json_5.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/coll

In [ ]:
#index_name = 'trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16'

In [ ]:
new_collections_path = remove_expansions_from_empty_docs(index_name)

['/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16/json_1.json', '/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16/json_2.json', '/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16/json_3.json', '/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16/json_4.json', '/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16/json_5.json', '/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16/json_6.json', '/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16/json_7.json', '/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16/json_8.json', '/conte

  0%|          | 0/11 [00:00<?, ?it/s]

In [55]:
new_collections_path, index_name

('/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries-expand-non-empty',
 'trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries')

In [56]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input {new_collections_path} \
  --index indexes/{index_name}-expand-non-empty \
  --generator DefaultLuceneDocumentGenerator \
  --threads 9 \
  --storePositions --storeDocvectors --storeRaw

2023-04-12 04:48:06,926 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-04-12 04:48:06,928 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-04-12 04:48:06,928 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-04-12 04:48:06,928 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: /content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries-expand-non-empty
2023-04-12 04:48:06,928 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-04-12 04:48:06,929 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2023-04-12 04:48:06,929 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 9
2023-04-12 04:48:06,929 INFO  [main]

In [57]:
!python -m pyserini.search.lucene \
  --index indexes/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 0.82 --b 0.68

Setting BM25 parameters: k1=0.82, b=0.68
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries-expand-non-empty.txt...
100% 50/50 [00:04<00:00, 10.78it/s]


In [ ]:
from evaluate import load

def eval(run_file):
  run = pd.read_csv(run_file, sep="\s+", 
                    names=["query", "q0", "docid", "rank", "score", "system"])
  run = run.to_dict(orient="list")

  trec_eval = load("trec_eval")
  results = trec_eval.compute(predictions=[run], references=[qrel])
  return results['NDCG@10']

In [59]:
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries-expand-non-empty.txt")

0.6444682838251563

In [ ]:
eval("/content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16-expand-non-empty.txt")

0.6472917171557704

### 2nd Test: Expand titles